In [1]:
import numpy as np
import pandas as pd
import sklearn.model_selection as ms
import sklearn.feature_extraction.text as text
import sklearn.naive_bayes as nb
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./leads.csv')

sub_df = pd.read_csv('./lead_notes.csv')

#merge하기
df_right = sub_df[['ParentObjectID', 'Text', 'Created_On']]
df_right['Created_On'] = pd.to_datetime(df_right['Created_On'])
result_df = pd.merge(df, df_right, left_on='ObjectID', right_on='ParentObjectID', how='inner')

#필요한 column만 추출하기
columns_to_select = ['Name', 'Source_Text', 'Status_Text',
'Start_Date', 'End_Date', 'Owner_Party_Name',
'Sales_Unit_Name', 'Sales_Territory_Name',
'ObjectID', 'Created_On', 'Note', 'Text']
df_final = result_df[columns_to_select]

/tmp/ipykernel_2124/1822583988.py:1: DtypeWarning: Columns (2,9,11,22,23,46,47,48,49,56,59,66,105,109,114,117,120,126,127,128,140,141,143,144,145,158,167,168,174,175,176,178,180) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./leads.csv')
/tmp/ipykernel_2124/1822583988.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sub_df = pd.read_csv('./lead_notes.csv')
/tmp/ipykernel_2124/1822583988.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right['Created_On'] = pd.to_datetime(df_right['Created_On'])


In [3]:
# 'ObjectID'로 그룹화하고 'Created_On'의 최댓값을 가지는 행 선택

df_gb = df_final.loc[df_final.groupby('ObjectID')['Created_On'].idxmin()]
df_gb.head()

#이제 이 코드를 보면 알겠지만, Converted인 데이터는 Text가 "Target Conversion Customer Node" 이런 내용임
df_gb[['ObjectID', 'Status_Text', 'Note', 'Text']].iloc[:100]

,ObjectID,Status_Text,Note,Text
101298,00163E092ED91ED79BDD7C0FDE76743F,Converted,Customer Emailed Matt Cole regarding a new hig...,Customer Emailed Matt Cole regarding a new hig...
101299,00163E092ED91ED79CA798C00B9074B9,Converted,Name: Contact_First_Name Contact_Last_Name\nEm...,Name: Contact_First_Name Contact_Last_Name\nEm...
101301,00163E092ED91ED79CAD62F1CEB62AFC,Unqualified,First name: Contact_First_Name\nLast name: Con...,First name: Contact_First_Name\nLast name: Con...
101302,00163E092ED91ED79CAD80EB7D72EBFB,Closed,8/1/2017: sent customer to a distributor\n\nMe...,8/1/2017: sent customer to a distributor\n\nMe...
101303,00163E092ED91ED79CAD954B4B73ACC9,Closed,First name: Contact_First_Name\nLast name: Con...,First name: Contact_First_Name\nLast name: Con...
...,...,...,...,...
101421,00163E092ED91ED7A997ABE35A3C449C,Converted,以下为旋转油封使用工况\n\n使用温度：50℃±5°\n使用压力：6-12bar 单一设备压...,以下为旋转油封使用工况\n\n使用温度：50℃±5°\n使用压力：6-12bar 单一设备压...
101423,00163E092ED91ED7A9A23F59F43E54D9,Converted,对TRAA00100-VCBV产品比较感兴趣！,对TRAA00100-VCBV产品比较感兴趣！
101425,00163E092ED91ED7A9A27B62CD0ED684,Closed,对TRYS0011924N011感兴趣，目前正在寻找该油封的产品信息，请及时与客户沟通！,对TRYS0011924N011感兴趣，目前正在寻找该油封的产品信息，请及时与客户沟通！
101426,00163E092ED91ED7A9A337D14BC79B81,Converted,该公司主要生产液压机、液压油缸，对特瑞堡密封比较感兴趣，请联系！,该公司主要生产液压机、液压油缸，对特瑞堡密封比较感兴趣，请联系！


In [11]:
df_gb[['Start_Date', 'End_Date', 'Created_On', 'ObjectID', 'Status_Text', 'Note', 'Text']].iloc[:100]

,Start_Date,End_Date,Created_On,ObjectID,Status_Text,Note,Text
101298,7/22/2017,8/21/2017,2017-07-22 14:43:30.887000+00:00,00163E092ED91ED79BDD7C0FDE76743F,Converted,Customer Emailed Matt Cole regarding a new hig...,Customer Emailed Matt Cole regarding a new hig...
101299,7/25/2017,8/24/2017,2017-07-25 13:28:51.040000+00:00,00163E092ED91ED79CA798C00B9074B9,Converted,Name: Contact_First_Name Contact_Last_Name\nEm...,Name: Contact_First_Name Contact_Last_Name\nEm...
101301,7/25/2017,8/24/2017,2017-07-25 18:59:15.192000+00:00,00163E092ED91ED79CAD62F1CEB62AFC,Unqualified,First name: Contact_First_Name\nLast name: Con...,First name: Contact_First_Name\nLast name: Con...
101302,7/25/2017,8/24/2017,2017-07-25 19:05:41.913000+00:00,00163E092ED91ED79CAD80EB7D72EBFB,Closed,8/1/2017: sent customer to a distributor\n\nMe...,8/1/2017: sent customer to a distributor\n\nMe...
101303,7/25/2017,8/24/2017,2017-07-25 19:10:35.522000+00:00,00163E092ED91ED79CAD954B4B73ACC9,Closed,First name: Contact_First_Name\nLast name: Con...,First name: Contact_First_Name\nLast name: Con...
...,...,...,...,...,...,...,...
101421,9/29/2017,10/29/2017,2017-09-29 02:26:54.007000+00:00,00163E092ED91ED7A997ABE35A3C449C,Converted,以下为旋转油封使用工况\n\n使用温度：50℃±5°\n使用压力：6-12bar 单一设备压...,以下为旋转油封使用工况\n\n使用温度：50℃±5°\n使用压力：6-12bar 单一设备压...
101423,9/29/2017,10/29/2017,2017-09-29 12:37:55.681000+00:00,00163E092ED91ED7A9A23F59F43E54D9,Converted,对TRAA00100-VCBV产品比较感兴趣！,对TRAA00100-VCBV产品比较感兴趣！
101425,9/29/2017,10/29/2017,2017-09-29 12:48:05.186000+00:00,00163E092ED91ED7A9A27B62CD0ED684,Closed,对TRYS0011924N011感兴趣，目前正在寻找该油封的产品信息，请及时与客户沟通！,对TRYS0011924N011感兴趣，目前正在寻找该油封的产品信息，请及时与客户沟通！
101426,9/29/2017,10/29/2017,2017-09-29 13:29:18.682000+00:00,00163E092ED91ED7A9A337D14BC79B81,Converted,该公司主要生产液压机、液压油缸，对特瑞堡密封比较感兴趣，请联系！,该公司主要生产液压机、液压油缸，对特瑞堡密封比较感兴趣，请联系！


In [6]:
df_gb.isnull().sum()

Name                      161
Source_Text                 0
Status_Text                 0
Start_Date                  0
End_Date                    0
Owner_Party_Name            0
Sales_Unit_Name         48663
Sales_Territory_Name    40215
ObjectID                    0
Created_On                  0
Note                     1400
Text                      423
dtype: int64

In [8]:
null_note_rows = df_gb[df_gb['Note'].isnull()]
null_note_rows[['ObjectID', 'Status_Text', 'Note', 'Text']].iloc[:100]

,ObjectID,Status_Text,Note,Text
101322,00163E092ED91ED7A2936DE6D4812FF6,Converted,NaN,Target Conversion Customer Node ID:00163E092ED...
101381,00163E092ED91ED7A58E27063055B032,Converted,NaN,Target Conversion Customer Node ID:00163E092ED...
101387,00163E092ED91ED7A686DA6959F43E1C,Converted,NaN,Target Conversion Customer Node ID:00163E092ED...
101623,00163E092ED91EE7A6BCE8A0F9CCF4B0,Converted,NaN,Target Conversion Customer Node ID:00163E092ED...
101624,00163E092ED91EE7A6BD5964052DF7EA,Converted,NaN,Target Conversion Customer Node ID:00163E092ED...
...,...,...,...,...
105228,00163E6332EA1ED8A58D428665D1A4A8,Converted,NaN,Target Conversion Customer Node ID:00163E6332E...
105252,00163E6332EA1ED8A5C45AC61101E0E8,Converted,NaN,NaN
105298,00163E6332EA1ED8B8941E306FA7BA8B,Converted,NaN,Target Conversion Customer Node ID:00163E6332E...
105409,00163E6332EA1EE8A097CCB97BA9FF39,Converted,NaN,Target Conversion Customer Node ID:00163E6332E...


In [9]:
null_note_rows = df_gb[df_gb['Text'].isnull()]
null_note_rows[['ObjectID', 'Status_Text', 'Note', 'Text']].iloc[:100]

,ObjectID,Status_Text,Note,Text
102758,00163E561F911ED8B2D7398199EEBE5A,Converted,NaN,NaN
102765,00163E561F911ED8B2DBCF6073A14C66,Converted,NaN,NaN
103659,00163E561F911EE8B5F523D7FAF62E0A,Converted,NaN,NaN
104059,00163E561FC41ED88FC3D2753DAB38D2,Converted,NaN,NaN
104908,00163E561FC41EE8B98EB7C4C2457C6B,Closed,NaN,NaN
...,...,...,...,...
117878,00163E908B501EDA85CA88A2212A2FB4,Converted,NaN,NaN
117879,00163E908B501EDA85CA88A2212A4FB4,Converted,NaN,NaN
117880,00163E908B501EDA85CA88A2212A6FB4,Converted,NaN,NaN
117881,00163E908B501EDA85CA88A2212A8FB4,Converted,NaN,NaN


In [15]:
df_gb['Source_Text'].value_counts()

Source_Text
Website Contact Form                        34328
Web Member Registration                     13604
Sales                                        6227
Webinar                                      5012
Trade Show                                   4466
Conference                                   4140
Chat-bot                                     2043
e-Catalog                                    1446
Relationship newsletter                      1320
Sealing Solutions Configurator               1006
Marketing Intelligence Analysis               787
TSS Landing Page                              593
WeChat                                        531
Roadshow                                      467
Google Ads                                    446
Relevant Behavior Upon Campaigns              384
CAD Service                                   377
Trelleborg Mailboxes                          356
LinkedIn                                      287
Variseal Selector                     

In [16]:
df_gb['Name'].value_counts()

Name
Web Request                           28137
Member Registration                    6561
Member Registration Upload             4479
Web Request H&M Website                1917
Electronic Catalog RfQ                 1393
                                      ...  
SC dispensing leads_20190409_03           1
SC dispensing leads_20190409_22           1
SC dispensing leads_20190409_23           1
SC dispensing leads_20190409_24           1
Leads from DIECASTING 2019_July_12        1
Name: count, Length: 11768, dtype: int64

In [17]:
df_gb['Start_Date'].value_counts()

Start_Date
2020-08-27    3151
8/27/2020     1336
5/23/2023      887
2021-05-20     799
2020-12-15     781
              ... 
2022-07-16       1
2022-06-18       1
2022-05-14       1
2022-04-03       1
2022-11-06       1
Name: count, Length: 3083, dtype: int64

In [18]:
df_gb['Owner_Party_Name'].value_counts()

Owner_Party_Name
Jeremy Pearson         15018
Michael Thomas          3076
Stephanie Osborn        1861
Margaret Stevens        1428
Marissa Nelson          1350
                       ...  
Daisy Gallegos             1
Marilyn Jackson PhD        1
Tommy Buck                 1
Susan Murray               1
Justin Russell             1
Name: count, Length: 1108, dtype: int64

In [19]:
df_gb['Sales_Unit_Name'].value_counts()

Sales_Unit_Name
TSS China                             4326
TSS Germany                           3846
TSS US Industrial                     2123
TSS India                             1729
TSS Sweden                            1552
                                      ... 
TSS AeroHub APAC Hong Kong               1
TSS Paso Robles                          1
TSS Business & Project Development       1
TSS AeroHub Fort Wayne                   1
TSS Tijuana                              1
Name: count, Length: 104, dtype: int64

In [20]:
df_gb['Sales_Territory_Name'].value_counts()

Sales_Territory_Name
I14 - Industrial Distribution               3236
SE1 - Sweden      Marketing Co              1534
BR - TSS Brazil                             1112
HM -TSS Healthcare & Medical Minneapolis    1103
MS - TSS East                                967
                                            ... 
CN3 - TSS Minhang                              1
BG18 - Oman                                    1
U78 - Clwyd                                    1
ATM - TSS AutoHub CN                           1
EMV - Emobility                                1
Name: count, Length: 397, dtype: int64

In [21]:
df_gb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78759 entries, 101298 to 32818
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   Name                  78598 non-null  object             
 1   Source_Text           78759 non-null  object             
 2   Status_Text           78759 non-null  object             
 3   Start_Date            78759 non-null  object             
 4   End_Date              78759 non-null  object             
 5   Owner_Party_Name      78759 non-null  object             
 6   Sales_Unit_Name       30096 non-null  object             
 7   Sales_Territory_Name  38544 non-null  object             
 8   ObjectID              78759 non-null  object             
 9   Created_On            78759 non-null  datetime64[ns, UTC]
 10  Note                  77359 non-null  object             
 11  Text                  78336 non-null  object             
dtypes: d

In [25]:
df_deployment = df_gb.drop(['Created_On', 'End_Date', 'ObjectID', 'Text'], axis=1)

df_deployment.to_csv('Short3_leads.csv', index=False)

In [26]:
temp = pd.read_csv('./Short3_leads.csv')
temp.head()

,Name,Source_Text,Status_Text,Start_Date,Owner_Party_Name,Sales_Unit_Name,Sales_Territory_Name,Note
0,High Speed Rotary Application,Sales,Converted,7/22/2017,Linda Burnett,TSS West,NCA - TSS West,Customer Emailed Matt Cole regarding a new hig...
1,Back-up Ring,Sales,Converted,7/25/2017,Robert Mcpherson,TSS West,NCA - TSS West,Name: Contact_First_Name Contact_Last_Name\nEm...
2,Holding Ring Samples,Sales,Unqualified,7/25/2017,Todd Conley,TSS West,NCA - TSS West,First name: Contact_First_Name\nLast name: Con...
3,Metric O-Rings,Sales,Closed,7/25/2017,Linda Burnett,TSS West,NCA - TSS West,8/1/2017: sent customer to a distributor\n\nMe...
4,Multiple Parts,Sales,Closed,7/25/2017,Cole Price,TSS West,NCA - TSS West,First name: Contact_First_Name\nLast name: Con...
